In [1]:
using OpenCL

In [2]:
b0 = 1
b1 = 5
sigma = 10
n = Int32(10_000)
nboots = Int32(1_000)

srand(37)
x = Float32.(rand(n))

epsilon = Float32.(sigma * randn(n))

y = b0 + b1 * x + epsilon

10000-element Array{Float32,1}:
  -6.08834 
 -13.1337  
  -0.993628
  -2.22474 
 -15.1408  
  26.9492  
   3.66461 
  13.0844  
   2.24964 
  -7.70494 
   3.14928 
  11.7378  
  24.4508  
   ⋮       
   7.75193 
   3.43734 
  -8.5505  
  -9.54199 
   7.29454 
   0.574152
   6.34216 
  17.8809  
  -0.782937
  -9.38895 
  29.2672  
   2.21033 

In [6]:
const kernel_src = open("bootbeta.cl") do f
   readstring(f)
end

"// n is the length of x, y\n__kernel void bootbeta(__global const float *x\n        , __global const float *y\n        , __global float *betas\n\t\t, int n\n        )\n{\n    // TODO: Do we call srand() ?\n    float sumx = 0.0;\n    float sumy = 0.0;\n    float sumx2 = 0.0;\n    float sumxy = 0.0;\n    for(int i = 0; i < n; i++)\n    {\n        iboot = rand() % *n;\n        sumx += x[iboot];\n        sumy += y[iboot];\n        sumx2 += x[iboot] * x[iboot];\n        sumxy += x[iboot] * y[iboot];\n    }\n    float nd = (float) *n;\n    float xbar = sumx / nd;\n    float ybar = sumy / nd;\n    float x2bar = sumx2 / nd;\n    float xybar = sumxy / nd;\n\n    float b1 = (xybar - xbar * ybar) / (x2bar - xbar * xbar);\n    float b0 = ybar - b1 * xbar;\n\n    int id = get_global_id(0);\n\n    betas[2 * id] = b0;\n    betas[2 * id + 1] = b1;\n}\n"

In [7]:
device, ctx, queue = cl.create_compute_context()

# Copy over to the device
xd = cl.Buffer(Float32, ctx, (:r, :copy), hostbuf = x)
yd = cl.Buffer(Float32, ctx, (:r, :copy), hostbuf = y)

# Column major array storing bootstrapped b0, b1
beta_d = cl.Buffer(Float32, ctx, :w, 2*nboots)

program = cl.Program(ctx, source = kernel_src) |> cl.build!
bootbeta_kernel = cl.Kernel(program, "bootbeta")

ERROR (unhandled task failure): MethodError: no method matching unsafe_string(::Ptr{Void})
Closest candidates are:
  unsafe_string(::ZMQ.Message) at /Users/clark/.julia/v0.5/ZMQ/src/ZMQ.jl:439
  unsafe_string(::Cstring) at c.jl:187
  unsafe_string(::Union{Ptr{Int8},Ptr{UInt8}}) at strings/basic.jl:56
  ...
 in macro expansion at /Users/clark/.julia/v0.5/OpenCL/src/context.jl:95 [inlined]
 in (::OpenCL.cl.##44#46)() at ./task.jl:360


LoadError: CLError(code=-11, CL_BUILD_PROGRAM_FAILURE)

In [5]:
queue(bootbeta_kernel, nboots, nothing
    , knots_clbuff
    , coefficients_clbuff
    , intermediate_clbuff
    )

ll::Vector{Float32} = cl.read(queue, beta_d)

LoadError: UndefVarError: bootbeta_kernel not defined